Basically, when it failed in the middle of a batch, it might have written out the boats already (due to an accidental generator instead of list comprehension).
This meant some duplicates. Luckily, its easy to tell, as boats are always consecutive for a day. So we just keep the last set of consecutive boats for each day.

In [1]:
import pandas as pd
import os

path = "C:\\ML_Software\\All_Results"

for root, dirs, files in os.walk(path):
    print(root, dirs)
    if len(dirs) > 1:
        continue
    for file in files:
        if file.endswith(".new.csv"):
            os.remove(os.path.join(root, file))
        if file.endswith(".csv") and not file.endswith(".new.csv"): 
            print(f"Processing {file}")
            df = pd.read_csv(os.path.join(root, file), parse_dates=['date'])

            date_positions = {}
            prev_date = None
            start_index = 0

            for i, current_date in enumerate(df['date']):
                if current_date != prev_date:
                    if prev_date is not None:
                        if prev_date not in date_positions:
                            date_positions[prev_date] = []
                        date_positions[prev_date].append((start_index, i-1))
                    start_index = i
                    prev_date = current_date
                
            if prev_date is not None:
                if prev_date not in date_positions:
                    date_positions[prev_date] = []
                date_positions[prev_date].append((start_index, len(df)-1))

            non_consecutive_dates = {}
            for date, ranges in date_positions.items():
                if len(ranges) > 1:
                    non_consecutive_dates[date] = ranges

            for date, ranges in non_consecutive_dates.items():
                rows = None
                for start, end in ranges:
                    rows = df.iloc[start:end+1] if rows is None else pd.concat([rows, df.iloc[start:end+1]])
                    print(f"  {start} - {end}. Length: {end-start+1}")
            
            # keep only the last range
            date_changes = df['date'] != df['date'].shift()
            df['group_id'] = date_changes.cumsum()
            df['max_group_id'] = df.groupby('date')['group_id'].transform('max')
            df_last_group = df[df['group_id'] == df['max_group_id']].copy()
            df_last_group.drop(columns=['group_id', 'max_group_id'], inplace=True)
            # save the last range
            df_last_group.to_csv(os.path.join(root, file), index=False)
            print(f"  Saved {file}")

            



C:\ML_Software\All_Results ['HPC', 'Local_1', 'Local_2']
C:\ML_Software\All_Results\HPC []
Processing boat_detections.csv


C:\Users\turner30\AppData\Local\Temp\ipykernel_20852\247819951.py:15: UserWarning: Parsing dates in %d/%m/%Y format when dayfirst=False (the default) was specified. Pass `dayfirst=True` or specify a format to silence this warning.
  df = pd.read_csv(os.path.join(root, file), parse_dates=['date'])


  Saved boat_detections.csv
Processing coverage.csv
  Saved coverage.csv
Processing orders.csv
  Saved orders.csv
C:\ML_Software\All_Results\Local_1 []
Processing boat_detections.csv
  71212 - 71401. Length: 190
  71505 - 71694. Length: 190
  71402 - 71429. Length: 28
  71695 - 71722. Length: 28
  71430 - 71504. Length: 75
  71723 - 71797. Length: 75
  Saved boat_detections.csv
Processing coverage.csv
  Saved coverage.csv
Processing orders.csv
  Saved orders.csv
C:\ML_Software\All_Results\Local_2 []
Processing boat_detections.csv
  Saved boat_detections.csv
Processing coverage.csv
  Saved coverage.csv
Processing orders.csv
  Saved orders.csv
